# Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

* Create a Jupyter Notebook called `database_engineering.ipynb` and use this to complete all of your Database Engineering work.

* Use Pandas to read your cleaned measurements and stations CSV data.

* Use the `engine` and connection string to create a database called `hawaii.sqlite`.

* Use `declarative_base` and create ORM classes for each table.

  * You will need a class for `Measurement` and for `Station`.

  * Make sure to define your primary keys.

* Once you have your ORM classes defined, create the tables in the database using `create_all`.

In [4]:
import os
import pandas as pd
import numpy

In [5]:
c_stations_file = os.path.join('Resources', 'cleaned_hawaii_stations.csv')
c_measurements_file = os.path.join('Resources', 'cleaned_hawaii_measurements.csv')

In [6]:
c_stations_df = pd.read_csv(c_stations_file)
c_measurements_df = pd.read_csv(c_measurements_file)

In [7]:
c_stations_df.head()

,st_id,st_name,st_latitude,st_longitude,st_elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [8]:
c_measurements_df.head()

,st_id,rec_date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [9]:
!rm hawaii.sqlite

In [10]:
# Dependencies and boilerplate

import sqlalchemy   
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Float, Integer, String, Date
from sqlalchemy.orm import relationship


from sqlalchemy.orm import Session


In [11]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [12]:
# Use `declarative_base` from SQLAlchemy to model the table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.

Base = declarative_base()

In [13]:
# Define a Station class

class Station(Base):
    __tablename__ = "station"
    
    st_id = Column(String, primary_key=True)
    st_name = Column(String)
    st_latitude = Column(Float)
    st_longitude = Column(Float)
    st_elevation= Column(Float)


In [14]:
class Measurement(Base):
    __tablename__ = "measurement"
    
    st_id = Column(String, primary_key=True)
    rec_date = Column(String, primary_key=True) # date the measurements are recorded
    prcp = Column(Float) # prcp
    tobs = Column(Float) #tobs
    

In [15]:
# Use `create_all` to create the station , measurement tables in the database
Base.metadata.create_all(engine)

In [16]:
stations_data = c_stations_df.to_dict(orient='records')
measurements_data = c_measurements_df.to_dict(orient='records')


In [17]:
# Data is just a list of dictionaries that represent each row of data
print(stations_data[:5])

[{'st_id': 'USC00519397', 'st_name': 'WAIKIKI 717.2, HI US', 'st_latitude': 21.2716, 'st_longitude': -157.8168, 'st_elevation': 3.0}, {'st_id': 'USC00513117', 'st_name': 'KANEOHE 838.1, HI US', 'st_latitude': 21.4234, 'st_longitude': -157.8015, 'st_elevation': 14.6}, {'st_id': 'USC00514830', 'st_name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'st_latitude': 21.5213, 'st_longitude': -157.8374, 'st_elevation': 7.0}, {'st_id': 'USC00517948', 'st_name': 'PEARL CITY, HI US', 'st_latitude': 21.3934, 'st_longitude': -157.9751, 'st_elevation': 11.9}, {'st_id': 'USC00518838', 'st_name': 'UPPER WAHIAWA 874.3, HI US', 'st_latitude': 21.4992, 'st_longitude': -158.0111, 'st_elevation': 306.6}]


In [18]:
# Data is just a list of dictionaries that represent each row of data
print(measurements_data[:5])

[{'st_id': 'USC00519397', 'rec_date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'st_id': 'USC00519397', 'rec_date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'st_id': 'USC00519397', 'rec_date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'st_id': 'USC00519397', 'rec_date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'st_id': 'USC00519397', 'rec_date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [19]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [20]:
# Save the reference to the `customers` table as a variable called `table`
table_station = sqlalchemy.Table('station', metadata, autoload=True)
table_measurement = sqlalchemy.Table('measurement', metadata, autoload=True)

In [21]:
# Use `table.delete()` to remove any pre-existing data.
conn.execute(table_station.delete())
conn.execute(table_measurement.delete())

In [22]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table_station.insert(), stations_data)
conn.execute(table_measurement.insert(), measurements_data)

In [23]:
# Test that the insert works by fetching the rows. 
conn.execute("select * from station").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [24]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[('USC00519397', '2010-01-01', 0.08, 65.0),
 ('USC00519397', '2010-01-02', 0.0, 63.0),
 ('USC00519397', '2010-01-03', 0.0, 74.0),
 ('USC00519397', '2010-01-04', 0.0, 76.0),
 ('USC00519397', '2010-01-07', 0.06, 70.0)]